<a href="https://colab.research.google.com/github/GRARLBD/FIA2023-TPI/blob/main/FIA2023_TPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
install.packages('lubridate')
install.packages('ggplot2')
install.packages('ggdist')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘distributional’, ‘numDeriv’, ‘quadprog’, ‘Rcpp’




In [ ]:
install.packages('neuralnet')
install.packages('NeuralNetTools')
install.packages('gridExtra')


In [ ]:
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("caret") #para el particionado del df
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

In [ ]:
library(lubridate)
library(ggplot2)
library(ggdist)
library(dplyr)
library('neuralnet')
library('NeuralNetTools')
library('gridExtra')
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

In [9]:
df_princess<- read.csv("/content/princess.csv")
df_wine <- read.csv("/content/wine.csv")

In [ ]:
table(df_princess$Cured)

In [ ]:
head(df_wine) # MOSTRAMOS LA PRIMERAS FILAS


In [ ]:
# Mostrar resumen estadístico
summary(df_wine)


In [ ]:
summary(df_princess)

In [ ]:
str(df_princess) # muestra la estructura del dataset

In [ ]:
cat("Nro de filas 'DF_WINE' : ", nrow(df_wine)) # muestra el número de filas en el data frame
# Vemos si hay campos nulos en alguna columna
cat("\nDatos nulos en cada columna: ' : ", colSums(is.na(df_wine)), "\n")

# Obtener índices de las filas duplicadas
indices_duplicados <- which(duplicated(df_wine) | duplicated(df_wine, fromLast = TRUE))

if (length(indices_duplicados) > 0) {
  print(paste("El conjunto de datos tiene", length(indices_duplicados), "filas duplicadas."))
  # Mostrar los índices de las filas duplicadas
  print("Índices de filas duplicadas:")
  print(indices_duplicados)
  # Mostrar las filas duplicadas
  print("Filas duplicadas:")
  print(df_wine[indices_duplicados, ])
} else {
  print("No se encontraron filas duplicadas en el conjunto de datos.")
}


In [10]:
# Eliminar filas duplicadas del conjunto de datos 'datos'
datos_sin_duplicados <- unique(df_wine)

# Mostrar la cantidad de filas antes y después de eliminar duplicados
print(paste("Número de filas antes de eliminar duplicados:", nrow(df_wine)))
print(paste("Número de filas después de eliminar duplicados:", nrow(datos_sin_duplicados)))


[1] "Número de filas antes de eliminar duplicados: 1599"
[1] "Número de filas después de eliminar duplicados: 1359"


In [10]:
cat("Distribución de los valores de la etiqueta\n")
table(datos_sin_duplicados$quality)

Distribución de los valores de la etiqueta



 bad good 
 640  719 

In [11]:
# Filtrar el conjunto de datos por etiquetas "good" y "bad"
datos_good <- subset(datos_sin_duplicados, datos_sin_duplicados$quality == "good")
datos_bad <- subset(datos_sin_duplicados, datos_sin_duplicados$quality == "bad")

# Seleccionar aleatoriamente 640 registros de cada etiqueta
datos_good_seleccionados <- datos_good[sample(nrow(datos_good), 640, replace = FALSE), ]
datos_bad_seleccionados <- datos_bad[sample(nrow(datos_bad), 640, replace = FALSE), ]

# Combinar los datos seleccionados de cada etiqueta en un nuevo conjunto de datos equilibrado
datos_equilibrados <- rbind(datos_good_seleccionados, datos_bad_seleccionados)

# Verificar la cantidad de filas por etiqueta en el conjunto de datos final
cat("DATASET EQUILIBRADO : \n")
print(table(datos_equilibrados$quality))


DATASET EQUILIBRADO : 

 bad good 
 640  640 


In [90]:
#PREDICION CON SVM

df_balanceado160SVM = datos_equilibrados
df_balanceado160SVM$quality <- as.factor(df_balanceado160SVM$quality)

# Función para normalización Min-Max
normalize_min_max <- function(x) {
  return((x - min(x)) / (max(x) - min(x)))
}

# Seleccionar solo las columnas numéricas para la normalización
columnas_numericas <- sapply(df_balanceado160SVM, is.numeric)
df_balanceado160SVM[columnas_numericas] <- lapply(df_balanceado160SVM[columnas_numericas], normalize_min_max)

# Inicializar un vector para almacenar las precisiones de cada iteración
precisiones <- numeric(100)

# Iterar 100 veces
for (i in 1:100) {
    # Código para particionar el conjunto de datos, entrenar y evaluar el modelo SVM
    set.seed(2023 + i) # Cambiar la semilla para cada iteración
    t.ids <- createDataPartition(df_balanceado160SVM$quality, p = 0.7, list = FALSE)
    modelo <- svm(quality ~ ., data = df_balanceado160SVM[t.ids, ], kernel = "radial")
    prediccion <- predict(modelo, df_balanceado160SVM[-t.ids, ])
    matriz_confusion <- table(df_balanceado160SVM$quality[-t.ids], prediccion, dnn = c('Real', 'Predicho'))
    confusion_matrix <- confusionMatrix(matriz_confusion)
    precisiones[i] <- confusion_matrix$overall['Accuracy']
}

# Mostrar la precisión promedio y otros detalles
cat("Precisión promedio (Accuracy):", mean(precisiones), "\n")
cat("Desviación estándar de la precisión:", sd(precisiones), "\n")
cat("\n{***** Detalles del entrenamiento del modelo SVM *****}\n\n")

confusion_matrix

Precisión promedio (Accuracy): 0.7470573 
Desviación estándar de la precisión: 0.01877036 

{***** Detalles del entrenamiento del modelo SVM *****}



Confusion Matrix and Statistics

      Predicho
Real   bad good
  bad  152   40
  good  61  131
                                          
               Accuracy : 0.737           
                 95% CI : (0.6899, 0.7803)
    No Information Rate : 0.5547          
    P-Value [Acc > NIR] : 1.223e-13       
                                          
                  Kappa : 0.474           
                                          
 Mcnemar's Test P-Value : 0.04658         
                                          
            Sensitivity : 0.7136          
            Specificity : 0.7661          
         Pos Pred Value : 0.7917          
         Neg Pred Value : 0.6823          
             Prevalence : 0.5547          
         Detection Rate : 0.3958          
   Detection Prevalence : 0.5000          
      Balanced Accuracy : 0.7398          
                                          
       'Positive' Class : bad             
                                          

In [91]:
#PREDICION CON RANDOM FOREST

df_rf= datos_equilibrados
df_rf$quality <- as.factor(df_rf$quality)

# Función para normalización Min-Max
normalize_min_max <- function(x) {
  return((x - min(x)) / (max(x) - min(x)))
}

# Seleccionar solo las columnas numéricas para la normalización
columnas_numericas <- sapply(df_rf, is.numeric)
df_rf[columnas_numericas] <- lapply(df_rf[columnas_numericas], normalize_min_max)

# Inicializar un vector para almacenar las precisiones de cada iteración
precisiones_rf <- numeric(100)

# Iterar 100 veces
for (i in 1:100) {
  # Establecer semilla para reproducibilidad en cada iteración
  set.seed(2023 + i)

  # Crear índices para dividir el conjunto de datos en entrenamiento (70%) y prueba (30%)
  t.ids <- sample(nrow(df_rf), 0.7 * nrow(df_rf))
  test.ids <- setdiff(1:nrow(df_rf), t.ids)
  modelo_rf <- randomForest(quality ~ ., data = df_rf[t.ids, ])

  prediccion_rf <- predict(modelo_rf, newdata = df_rf[test.ids, ])
  matriz_confusion <- table(prediccion_rf, df_rf$quality[test.ids], dnn = c('Real' ,'Predicho'))
  confusion_matrix <- confusionMatrix(matriz_confusion)
  precisiones_rf[i] <- confusion_matrix$overall['Accuracy']
}

# Mostrar la precisión promedio y otros detalles
cat("Precisión promedio (Accuracy) del modelo de Random Forest:", mean(precisiones_rf), "\n")
cat("Desviación estándar de la precisión:", sd(precisiones_rf), "\n")
cat("\n{***** Detalles del entrenamiento del modelo de Random Forest *****}\n\n")

confusion_matrix


Precisión promedio (Accuracy) del modelo de Random Forest: 0.763724 
Desviación estándar de la precisión: 0.01997336 

{***** Detalles del entrenamiento del modelo de Random Forest *****}



Confusion Matrix and Statistics

      Predicho
Real   bad good
  bad  144   48
  good  52  140
                                          
               Accuracy : 0.7396          
                 95% CI : (0.6926, 0.7828)
    No Information Rate : 0.5104          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.4792          
                                          
 Mcnemar's Test P-Value : 0.7642          
                                          
            Sensitivity : 0.7347          
            Specificity : 0.7447          
         Pos Pred Value : 0.7500          
         Neg Pred Value : 0.7292          
             Prevalence : 0.5104          
         Detection Rate : 0.3750          
   Detection Prevalence : 0.5000          
      Balanced Accuracy : 0.7397          
                                          
       'Positive' Class : bad             
                                          

In [1]:
rm(list = ls())


In [86]:
#PREDICION CON REDES NEURONALES

# Crear una copia de 'datos_equilibrados' con solo las columnas deseadas
df_redes_neuronales <- datos_equilibrados

# Convertir 'quality' a valores numéricos (por ejemplo, 'good' a 1 y 'bad' a 0)
df_redes_neuronales$quality <- ifelse(df_redes_neuronales$quality == "good", 1, 0)

# Función para normalización Min-Max
normalize_min_max <- function(x) {
  return((x - min(x)) / (max(x) - min(x)))
}

# Seleccionar solo las columnas numéricas para la normalización
columnas_numericas <- sapply(df_redes_neuronales, is.numeric)
df_redes_neuronales[columnas_numericas] <- lapply(df_redes_neuronales[columnas_numericas], normalize_min_max)

# Establecer semilla para reproducibilidad
set.seed(123)

# Número de filas en el conjunto de datos
num_filas <- nrow(df_redes_neuronales)

# Índices para conjunto de entrenamiento (70%) y prueba (30%)
indice_entrenamiento <- sample(1:num_filas, 0.7 * num_filas, replace = FALSE)
indice_prueba <- setdiff(1:num_filas, indice_entrenamiento)

# Crear conjuntos de datos de entrenamiento y prueba
datos_entrenamiento <- df_redes_neuronales[indice_entrenamiento, ]
datos_prueba <- df_redes_neuronales[indice_prueba, ]


red <- neuralnet(
  quality ~ ., # Fórmula (ajusta a tus nombres de columnas)
  data = datos_entrenamiento, # Conjunto de datos
  hidden=10, #capas ocultas
  linear.output = FALSE, #si act.fct NO debe aplicarse a la salida, TRUE, de lo contrario como FALSE.
  act.fct="logistic", #función de activación
  algorithm="backprop", #tipo de algoritmo
  threshold=0.2, #umbral
  learningrate=0.005, #tasa de aprendizaje
  rep=100 #cantidad de repeticiones
)

# Calcular las salidas de la red neuronal para los datos de prueba
resultados <- compute(red, datos_prueba[, -ncol(datos_prueba)])

In [87]:
# Obtener las predicciones para los datos de prueba
predicciones <- ifelse(resultados$net.result > 0.5, 1, 0)

# Calcular la precisión manualmente
precision <- sum(predicciones == datos_prueba$quality) / length(predicciones)

# Mostrar la precisión
cat("Precisión en datos de prueba:", precision, "\n")


Precisión en datos de prueba: 0.7213542 
